In [1]:
import rg
from rg.diagrams import diagram, composition_diagram, diagram_set
from rg.interaction import interaction as J 
from rg.interaction import interaction_identity
from rg.interaction import interaction_system
from rg.interaction import composite_interaction as G
from rg.theory import ftheory

# Example representations
Here we show the matrix form for adding fields for arbitrary number of species (1 row per species). We display the latex symbols and also the diagrams

The diagrams are of the residuals so loops/internal structure are not shown here (see the composite examples later where we look at internal structure). 

The & operator or * operator (todo) are both used as diagram products allowing diagrams to be merged

In [2]:
j = J([[1,1], 
       [2,0]])
j

$\phi_0 \phi_1^{2} \tilde{\phi}_0$

In [3]:
j.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-45.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(45.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [4]:
transmutation = J([[1,0],[0,1]])
transmutation.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [5]:
coagulation = J([[2,1],[0,0]])
coagulation.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [6]:
branching = J([[1,2],[0,0]])
branching.diagram
#branching

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [7]:
branching.tensor

array([[1, 2],
       [0, 0]])

In [8]:
(branching&coagulation).diagram

symmetry factor: 2


<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [9]:
(coagulation&branching).diagram

symmetry factor: 2


<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [10]:
(branching&j).diagram

symmetry factor: 2


<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-45.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(45.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

# Dimensional Analysis
For dimensional analysis one construct a system of terms, each term is an interaction J.

One proposes known coupling values e.g. known dimensions of propagators 

the ftheory class permutes all non fixed couplings, proposing that permutations P(n,k) are dimensionless - each permutation creates a theory

A theory fixes the dimension of fields and all couplings. The theories can then be compared and chcked via the ftheory class

In [11]:
#blank -> tilde
#BRW0
interactions = [
 J([[1,1],[0,0]]),
 J([[0,0],[1,1]]),
 J([[1,0],[0,1]]), #tau 
 J([[1,2],[0,0]]), #s
 J([[1,1],[0,1]]), #quasi-branch sigma
 J([[1,1],[1,1]]), #kappa
 J([[1,0],[1,1]]), #lambda
 ]

#BWS1 - #add two more terms chi and eta
interactions = interactions +  [
 J([[0,1],[1,2]]), #
 J([[1,0],[1,2]]), #   
]

#VRS - add two more terms annhilation
interactions = interactions +  [
 J([[1,1],[1,0]]), #
 J([[2,1],[0,1]]), #   
]

#display
[j.display() for j in interactions]

⎡                                                                             
⎣\phi₀⋅\tilde{\phi}₀, \phi₁⋅\tilde{\phi}₁, \phi₀⋅\tilde{\phi}₁, \phi₀⋅\tilde{\

     2                                                                        
phi}₀ , \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁, \phi₀⋅\phi₁⋅\tilde{\phi}₀⋅\tilde{\p

                                                                  2           
hi}₁, \phi₀⋅\phi₁⋅\tilde{\phi}₁, \phi₁⋅\tilde{\phi}₀⋅\tilde{\phi}₁ , \phi₀⋅\ph

                2                                  2                          
i₁⋅\tilde{\phi}₁ , \phi₀⋅\phi₁⋅\tilde{\phi}₀, \phi₀ ⋅\tilde{\phi}₀⋅\tilde{\phi

  ⎤
}₁⎦

In [12]:
Lambda = rg.T**(-1) # L**(-1*dim)
new_couplings = {J([ [0,0],  [1,1]] ): Lambda, #[field]*lambda is the inverse measure
                 J([ [1,1],  [0,0]] ): Lambda, }

#example theory
FT = ftheory(new_couplings, 
             dimensionless=[J([ [1,2],  [0,0]] ), 
                            J([ [1,1],  [0,1]] )  ])
FT.display()

⎡                                   1⎤
⎢       \phi₁⋅\tilde{\phi}₁         ─⎥
⎢                                   T⎥
⎢                                    ⎥
⎢                                   1⎥
⎢       \phi₀⋅\tilde{\phi}₀         ─⎥
⎢                                   T⎥
⎢                                    ⎥
⎢                         2          ⎥
⎢      \phi₀⋅\tilde{\phi}₀          1⎥
⎢                                    ⎥
⎣\phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁  1⎦

In [13]:
FT.interpret_dimensions(interactions)

⎡                                            -d  ⎤
⎢          \phi₀⋅\tilde{\phi}₀              L    ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢          \phi₁⋅\tilde{\phi}₁              L    ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢          \phi₀⋅\tilde{\phi}₁              L    ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢                            2              L    ⎥
⎢         \phi₀⋅\tilde{\phi}₀               ───  ⎥
⎢                                            T   ⎥
⎢                                                ⎥
⎢                                            -d  ⎥
⎢                                           L    ⎥
⎢   \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁       ───  ⎥
⎢                                            T   ⎥
⎢                                                ⎥
⎢                              

In [14]:
FT.interpret_couplings(interactions, l_power_dim=4)

⎡                                         1 ⎤
⎢          \phi₀⋅\tilde{\phi}₀            ──⎥
⎢                                          2⎥
⎢                                         L ⎥
⎢                                           ⎥
⎢                                         1 ⎥
⎢          \phi₁⋅\tilde{\phi}₁            ──⎥
⎢                                          2⎥
⎢                                         L ⎥
⎢                                           ⎥
⎢                                         1 ⎥
⎢          \phi₀⋅\tilde{\phi}₁            ──⎥
⎢                                          2⎥
⎢                                         L ⎥
⎢                                           ⎥
⎢                            2              ⎥
⎢         \phi₀⋅\tilde{\phi}₀             1 ⎥
⎢                                           ⎥
⎢   \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁     1 ⎥
⎢                                           ⎥
⎢                                          2⎥
⎢\phi₀⋅\phi₁⋅\tilde{\phi}₀⋅\tilde{

In [15]:
#we can create a bunch of theories by permuting dimensionless couplings given the input known field dimensions and all terms
cs  = ftheory.theories(interactions, new_couplings)

In [16]:
#we can display a matrix for all theories for the values of the fields \phi_0, \tilde{\phi}_0, \phi_1, \tilde{\phi_1}
ftheory.matrices(cs).general_form

⎡ -d + 2      -2      -d + 4     -4  ⎤
⎢                                    ⎥
⎢   -d        0       -d + 2     -2  ⎥
⎢                                    ⎥
⎢   -2      -d + 2      0        -d  ⎥
⎢                                    ⎥
⎢   -d        0       -d + 2     -2  ⎥
⎢                                    ⎥
⎢   -4      -d + 4      -2     -d + 2⎥
⎢                                    ⎥
⎢ -d + 2      -2      -d + 2     -2  ⎥
⎢                                    ⎥
⎢ -d + 2      -2        -d       0   ⎥
⎢                                    ⎥
⎢ -d + 2      -2     -2⋅d + 4  d - 4 ⎥
⎢                                    ⎥
⎢ -d + 2      -2        -2     -d + 2⎥
⎢                                    ⎥
⎢ -d + 2      -2     -2⋅d + 4  d - 4 ⎥
⎢                                    ⎥
⎢   -2      -d + 2    -d + 2     -2  ⎥
⎢                                    ⎥
⎢   -d        0       -d + 2     -2  ⎥
⎢                                    ⎥
⎢   0         -d      -d + 2     -2  ⎥
⎢                        

In [17]:
#and we can evalaute the values at a certain d, last column is the inverse measure L^dT^1 with T=L^2 at d=d_c
ftheory.matrices(cs).criterion(4)

⎡2  2  0  4  -6⎤
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢2  2  0  4  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢0  4  2  2  -6⎥
⎢              ⎥
⎢0  4  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢2  2  2  2  -6⎥
⎢              ⎥
⎢2  2  4  0  -6⎥
⎢              ⎥
⎢4  4  6  2  -6⎥
⎢              ⎥
⎢4  0  2  2  -6⎥
⎢              ⎥
⎢4  4  6  2  -6⎥
⎢              ⎥
⎢0  4  2  2  -6⎥
⎢              ⎥
⎣0  4  2  2  -6⎦

# Composing and displaying diagrams
Work in progress. Lots of different ways to render things - starting with the basics, the ones that make development clearer.

In [45]:
G(branching)*G(coagulation)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill='green' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="41" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="59" x2="38" y2="59" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>

In [18]:
G(branching)*G(transmutation)*G(j)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="50" r="2" stroke="orange" stroke-width="1" fill='orange' /> <circle cx="38" cy="32" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="38" cy="68" r="2" stroke="orange" stroke-width="1"  /> <line x1="102" y1="41" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="41" x2="88" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="59" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="59" x2="38" y2="32" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="32" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="50" x2="70" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="50" x2="38" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"  stroke-dasharray='2,2' /><line x1="138" y1="50" x2="120" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="68" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>

In [19]:
diagram_set([G(branching)*G(transmutation),
             G(branching)*G(coagulation),
             G(branching)*G(transmutation)*G(j)])

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="210">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="41" x2="38" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="52" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="41" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="52" y1="59" x2="38" y2="59" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray=\'2,2\' /><line x1="88" y1="50" x2="70" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="100" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="32" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="50" r="2" stroke="orange" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' />

In [20]:
 (G(branching)*G(transmutation)*G(j)).residual_interaction.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

## Permutations

In [44]:
import itertools,functools

def _combine(set_of_interactions):
        def _merge(a,b): 
            if a is None:return b
            if b is None:return a
            return a * b
        return functools.reduce(_merge, set_of_interactions, None)
    
def graph_permutations(primitives, loop_orders = [0], max_k=3):
    if not isinstance(loop_orders, list):loop_orders = [loop_orders]
    l = [] 
    
    for i in range(2,max_k+1):
        for tup in list(itertools.permutations(primitives,i)):
            res = _combine(list(tup))
            l.append(res)
    #todo : define uniqueness and validity
    return list(set(l))

#simple sample system
res = graph_permutations([G(branching), G(transmutation),G(coagulation),G(j)])
#our vicious walker system
res = graph_permutations([G(j) for j in interactions])

len(res)

443

In [43]:
diagram_set([G(j) for j in interactions])

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="610">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="50" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="50" r="2" stroke="orange" stroke-width="1"  /> <line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="100" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="150" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="2" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="200" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="250" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="59"

In [ ]:
#IS.__display_interaction_dims__()

# Additional tree-level diagrams

In [ ]:
#do permutations of some order with an and-reduction plus validation. if the pairings greater than 1, set is loop and dont yield for this order
compound_interaction(branching,coagulation)

In [ ]:
(j&transmutation&branching).diagram

In [ ]:
prims = [j, transmutation, coagulation, branching]
L = compound_interaction.combinations(prims, loop_orders=[0])
L[0]

# All 1-loop diagrams

In [ ]:
compound_interaction(coagulation, branching)
#once we figure out the integrals we can give them values and do z factors

In [ ]:
prims = [j, transmutation, coagulation, branching]
L = compound_interaction.combinations(prims, loop_orders=[1])
L[0]